# F1 statistics & simulation

## Introduction
Jupyter notebook created by David Chevrier, Diggers  
Built on ActivePivot Python library, Community version  
http://diggers-consulting.com  
contact@diggers-consulting.com  
March 2020

## Data source
Data from https://ergast.com/mrd/db/#csv or https://www.kaggle.com/draeg82/exploration-of-f1-dataset/data  
Race data from 1950 to 2019


## 1. Initialization & creation of Activepivot session

In [1]:
import activeviam as av
import numpy as np
import pandas as pd

session = av.create_session()

## 2. Creation of Stores

In [2]:
# types definition to correctly import data in stores
resultsTypes = {
    "points": av.types.DOUBLE,
}

In [4]:
sDrivers = session.read_csv("./drivers.csv", keys=["driverId"], store_name="F1 drivers")

In [7]:
#sRaces = session.read_csv("./races.csv", keys=['raceId','circuitId','year'], store_name="F1 races")
sRaces = session.read_csv("./races.csv", keys=['raceId'], store_name="F1 races")

In [8]:
#sResults = session.read_csv("./results.csv", keys=['resultId','raceId','driverId'], store_name="F1 results", types=resultsTypes)
sResults = session.read_csv("./results.csv", keys=['resultId'], store_name="F1 results", types=resultsTypes)

In [ ]:
sDriverStandings = session.read_csv("./driver_standings.csv", keys=["driverStandingsId"], store_name="F1 driver standings")

In [14]:
print('Number of results: ',sResults.shape,'\nNumber of driver_standings: ',sDriverStandings.shape)

Number of results:  {'rows': 24614, 'columns': 18} 
Number of driver_standings:  {'rows': 32342, 'columns': 7}


In [15]:
sResults.join(sDrivers,mapping={"driverId":"driverId"})
sResults.join(sRaces, mapping={"raceId": "raceId"})
#sResults.head(joined_columns=True)

In [ ]:
#sResults.head(joined_columns=True).columns

In [10]:
# joins between stores
sDriverStandings.join(sDrivers,mapping={"driverId":"driverId"})
sDriverStandings.join(sRaces, mapping={"raceId": "raceId"})

In [11]:
#load_all_data necessary otherwise stores are loaded with 10000 lines max
session.load_all_data()

## 3. Cube

### Cube for Race Rresults

In [16]:
f1cube= session.create_cube(sResults,"F1Cube")

In [17]:
l = f1cube.levels
m = f1cube.measures
h = f1cube.hierarchies

In [18]:
session.url

'http://localhost:58296'

### Cube for driver standings

In [96]:
f1stdcube= session.create_cube(sDriverStandings,"F1StdCube")
ls = f1stdcube.levels
ms = f1stdcube.measures
hs = f1stdcube.hierarchies

## 4. First data visualization

### dataviz1
A simple data visualization showing a table with the total number of races by driver, sorted by descending 'count' field

In [20]:
f1cube.visualize('Total number of races by driver')

Install the ActiveViam JupyterLab extension to see this widget.

## 5 Measures and first queries

In [21]:
# Definition of the measure aggregating the number of points on 2 particular levels: races and drivers
m['Total Points']=av.agg.sum(m['points.SUM'], on = ('driverId','raceId'))

### query1
A simple query that returns a dataframe with the total number of points aggregated by driver forname,surname

In [101]:
dfq1=f1cube.query(m['Total Points'],levels=[l["driverForename"],l["driverSurname"]])
dfq1

,,Total Points
driverForename,driverSurname,
Adolf,Brudes,0.0
Adolfo,Cruz,0.0
Adrian,Sutil,124.0
Adrián,Campos,0.0
Aguri,Suzuki,8.0
...,...,...
Zsolt,Baumgartner,1.0
Élie,Bayol,2.0
Éric,Bernard,10.0


### query2
A similar query with the addition of the condition parameters, used as a filter on levels (not possible as of today on the measure)

In [23]:
dfq2=f1cube.query(m['Total Points'],levels=[l["driverForename"],l["driverSurname"]],condition=l["driverSurname"]=="Prost")
dfq2
# check data here: https://www.statsf1.com/en/alain-prost.aspx

,,Total Points
driverForename,driverSurname,
Alain,Prost,798.5


### query3
Another query returning a datafram aggregating the Total points mesaure by driver and by year

In [24]:
#dfq3 = f1cube.query(m['Total Points'],levels=[l["driverRef"],l["driverForename"],l["driverSurname"],l["year"]])
dfq3 = f1cube.query(m['Total Points'],levels=[l["driverRef"],l["year"]])
dfq3

Total Points
driverRef year              
Cannoc    1971           0.0
Changy    1959           0.0
abate     1963           0.0
abecassis 1951           0.0
          1952           0.0
...                      ...
zorzi     1976           0.0
          1977           1.0
zunino    1979           0.0
          1980           0.0
          1981           0.0

[3096 rows x 1 columns]

In [102]:
# You can then manipulate your dataframe like any other pandas dataframe, applying filtrer for example
#type(dfq3)
#dfq3.keys
dfq3[dfq3['Total Points']>0]

Total Points
driverRef  year              
adamich    1972           3.0
           1973           3.0
agabashian 1953           1.5
alan_brown 1952           2.0
albers     2005           4.0
...                       ...
wurz       2005           6.0
           2007          13.0
zanardi    1993           1.0
zonta      2000           3.0
zorzi      1977           1.0

[1424 rows x 1 columns]

In [27]:
# in this case the resulting dataframe is multiindexed
dfq3.loc["alesi"].loc[1990]['Total Points']

13.0

### dataviz2
Data visualization of the top-5 drivers with the highest total of points in their whole career

In [29]:
## check here in the widget configuration the "TopCount" filter used to select only the top-5
## check here the cell metadata to sort the data in the chart
#     "plotly": {
#         "layout": {
#             "yaxis": {
#                 "categoryorder": "total ascending"
#             }
#         }
#     },
f1cube.visualize('TOP-5 drivers with highest total points in career')

Install the ActiveViam JupyterLab extension to see this widget.

In [176]:
# preparing the world champions dataframe 
dfwc = pd.DataFrame(index=range(1950,2020),columns=['driverRef','Total Points'])

In [97]:
f1stdcube.query(ms["contributors.COUNT"],levels=[ls["year"],ls["raceId"],ls["driverRef"]])

contributors.COUNT
year raceId driverRef                    
1950 833    bira                        1
            cabantous                   1
            chiron                      1
            claes                       1
            crossley                    1
...                                   ...
2020 1031   ricciardo                   1
            russell                     1
            sainz                       1
            stroll                      1
            vettel                      1

[32342 rows x 1 columns]

In [177]:
for i in range(1950,2020):
    dfstd=f1stdcube.query(ms["contributors.COUNT"],levels=[ls["year"],ls["raceId"],ls["driverRef"],ls["position"]],condition=(ls["position"]=="1") & (ls["year"]==str(i)))
    dfwc.loc[i]['driverRef']=dfstd.loc[i].loc[dfstd.index.get_level_values('raceId').unique().max()].index.get_level_values('driverRef').tolist()[0]
#dfwc

,driverRef,Total Points
1950,farina,NaN
1951,fangio,NaN
1952,ascari,NaN
1953,ascari,NaN
1954,fangio,NaN
...,...,...
2015,hamilton,NaN
2016,rosberg,NaN
2017,hamilton,NaN
2018,hamilton,NaN


## Simulations
Variation of points scoring rule in F1 history  
[EN version](https://en.wikipedia.org/wiki/List_of_Formula_One_World_Championship_points_scoring_systems)  
[FR version](https://fr.wikipedia.org/wiki/Classement_des_pilotes_de_Formule_1_par_nombre_de_points#%C3%89volution_de_l'attribution_des_points_au_cours_du_temps)

### Preparation of a dataframe for the different scoring systems

In [178]:
# index = race year, 1 column for each position, values are the number of points scored for the race position
scoring_columns = np.arange(1,41)
#scoring_index = np.arange(1950,2019)
scoring_index = ('sc1950to1959fl sc1960 sc1961to1990 sc1991to2002 sc2003to2009 sc2010to2013 sc2014lr sc2015to2018 sc2019fl').split()

In [179]:
dfscoring = pd.DataFrame(0,index=scoring_index,columns=scoring_columns)

In [180]:
# before 1991, the calculation rule is not a simple aggregation given that only the Nth best results were retained
# dfscoring.loc['sc1950to1959fl'][1,2,3,4,5] = [8,6,4,3,2] #fastest lap bonus +1 point
# dfscoring.loc['sc1960'][1,2,3,4,5,6] = [8,6,4,3,2,1]
# dfscoring.loc['sc1961to1990'][1,2,3,4,5,6] = [9,6,4,3,2,1]
dfscoring.loc['sc1991to2002'][1,2,3,4,5,6] = [10,6,4,3,2,1]
dfscoring.loc['sc2003to2009'][1,2,3,4,5,6,7,8] = [10,8,6,5,4,3,2,1]
dfscoring.loc['sc2010to2013'][1,2,3,4,5,6,7,8,9,10] = [25,18,15,12,10,8,6,4,2,1]
# dfscoring.loc['sc2014lr'][1,2,3,4,5,6,7,8,9,10] = [25,18,15,12,10,8,6,4,2,1] #last race bonus double points
dfscoring.loc['sc2015to2018'] = dfscoring.loc['sc2010to2013']
# dfscoring.loc['sc2019fl'] = dfscoring.loc['sc2010to2013'] #fastest lap bonus +1 point

In [181]:
#dfscoring.loc['sc2015to2018'][1]
dfscoring

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
sc1950to1959fl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sc1960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sc1961to1990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sc1991to2002,10,6,4,3,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sc2003to2009,10,8,6,5,4,3,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
sc2010to2013,25,18,15,12,10,8,6,4,2,1,...,0,0,0,0,0,0,0,0,0,0
sc2014lr,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sc2015to2018,25,18,15,12,10,8,6,4,2,1,...,0,0,0,0,0,0,0,0,0,0
sc2019fl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### simulation on points

In [182]:
pointssystem_sim = f1cube.setup_simulation('pointssystem_sim', per=[l["positionText"]], replace=[m["points.SUM"]], base_scenario_name = 'Base')

In [ ]:
#del f1cube.simulations['pointssystem_sim']
#del f1cube.simulations['pointssystem_sim'].scenarios["System 2015 to 2018"]
#f1cube.simulations['pointssystem_sim']

In [183]:
sc2015to2018_scenario = pointssystem_sim.scenarios['System 2015 to 2018']
sc1991to2002_scenario = pointssystem_sim.scenarios['System 1991 to 2002']
sc2003to2009_scenario = pointssystem_sim.scenarios['System 2003 to 2009']

In [184]:
for i in range(1,11):
    x=float(dfscoring.loc["sc2015to2018"][i])
    sc2015to2018_scenario += (str(i), x, av.simulation.Priority.CRITICAL)
    
    x=float(dfscoring.loc["sc1991to2002"][i])
    sc1991to2002_scenario += (str(i), x, av.simulation.Priority.CRITICAL)
    
    x=float(dfscoring.loc["sc2003to2009"][i])
    sc2003to2009_scenario += (str(i), x, av.simulation.Priority.CRITICAL)

In [ ]:
#sc2015to2018_scenario.head(10)
sc1991to2002_scenario.head(10)
#sc2003to2009_scenario.head(10)

In [185]:
# query4bis
# Construction of the champonship winners dataframe base on scenario
dfq4b = f1cube.query(m['Total Points'],levels=[l["year"],l["pointssystem_sim"],l["driverRef"]])
dfq4b

Total Points
year pointssystem_sim    driverRef                 
1950 Base                ader                   0.0
                         agabashian             0.0
                         ascari                11.0
                         banks                  0.0
                         bettenhausen           1.0
...                                             ...
2019 System 2015 to 2018 ricciardo             54.0
                         russell                0.0
                         sainz                 96.0
                         stroll                21.0
                         vettel               238.0

[12384 rows x 1 columns]

In [186]:
# preparing the world champions dataframe comparison between scenarios
#dfwc_comparison = pd.DataFrame(index=range(1950,2020),columns=['Base Champion','Base Total Points','sc2015to2018 Champion','sc2015to2018 Total Points','sc2015to2018'])
dfwc_comparison = pd.DataFrame(index=range(1950,2020),columns=['Base Champion','sc2015to2018 Champion','sc2015to2018','sc1991to2002 Champion','sc1991to2002','sc2003to2009 Champion','sc2003to2009'])

In [187]:
#### "BASE" CALCULATIONS ARE INCORRECT for championships before 1991 because not all results were considered...
# Example: in 1988, only the 11 best results were considered, meaning that Senna became WC, even if Prost had scored more points...
dfq4b.loc[1988].loc['Base'].sort_values(by=['Total Points'], ascending=False)

,Total Points
driverRef,
prost,105.0
senna,94.0
berger,41.0
boutsen,27.0
alboreto,24.0
piquet,22.0
capelli,17.0
warwick,17.0
mansell,12.0


In [190]:
for i in range(1950,2020):
    #xbase = dfq4b.loc[i].loc['Base'].sort_values(by=['Total Points'], ascending=False).iloc[0].head(1)
    dfwc_drv = dfwc.loc[i]['driverRef']
    xsim1 = dfq4b.loc[i].loc['System 2015 to 2018'].sort_values(by=['Total Points'], ascending=False).iloc[0].head(1)
    xsim2 = dfq4b.loc[i].loc['System 1991 to 2002'].sort_values(by=['Total Points'], ascending=False).iloc[0].head(1)
    xsim3 = dfq4b.loc[i].loc['System 2003 to 2009'].sort_values(by=['Total Points'], ascending=False).iloc[0].head(1)
    #dfwc_comparison.loc[i]=[xbase.name,xsim1.name,xbase.name == xsim1.name,xsim2.name,xbase.name == xsim2.name,xsim3.name,xbase.name == xsim3.name]
    dfwc_comparison.loc[i]=[dfwc_drv,xsim1.name,dfwc_drv == xsim1.name,xsim2.name,dfwc_drv == xsim2.name,xsim3.name,dfwc_drv == xsim3.name]
#dfwc_comparison[dfwc_comparison['Base Champion']=="prost"]
dfwc_comparison

,Base Champion,sc2015to2018 Champion,sc2015to2018,sc1991to2002 Champion,sc1991to2002,sc2003to2009 Champion,sc2003to2009
1950,farina,farina,True,farina,True,fagioli,False
1951,fangio,fangio,True,fangio,True,fangio,True
1952,ascari,ascari,True,ascari,True,ascari,True
1953,ascari,ascari,True,ascari,True,ascari,True
1954,fangio,fangio,True,fangio,True,fangio,True
...,...,...,...,...,...,...,...
2015,hamilton,hamilton,True,hamilton,True,hamilton,True
2016,rosberg,rosberg,True,rosberg,True,rosberg,True
2017,hamilton,hamilton,True,hamilton,True,hamilton,True
2018,hamilton,hamilton,True,hamilton,True,hamilton,True


In [205]:
# list of different WC using sc2015to2018 vs. Base
# dfwc_comparison[dfwc_comparison['sc2015to2018']==False]
dfwc_comparison.loc[range(1950,2020)][['Base Champion','sc2015to2018 Champion']][dfwc_comparison['sc2015to2018']==False]

,Base Champion,sc2015to2018 Champion
1964,surtees,hill
1965,clark,hill
1976,hunt,lauda
1984,lauda,prost
1988,senna,prost
1994,michael_schumacher,damon_hill
1997,villeneuve,michael_schumacher
1999,hakkinen,irvine


In [203]:
# list of different WC using sc1991to2002 vs. Base
dfwc_comparison.loc[range(1950,2020)][['Base Champion','sc1991to2002 Champion']][dfwc_comparison['sc1991to2002']==False]

,Base Champion,sc1991to2002 Champion
1964,surtees,hill
1984,lauda,prost
1988,senna,prost
2008,hamilton,massa


In [204]:
# list of different WC using sc2003to2009 vs. Base
dfwc_comparison.loc[range(1950,2020)][['Base Champion','sc2003to2009 Champion']][dfwc_comparison['sc2003to2009']==False]

,Base Champion,sc2003to2009 Champion
1950,farina,fagioli
1964,surtees,hill
1965,clark,hill
1970,rindt,ickx
1976,hunt,lauda
1984,lauda,prost
1988,senna,prost
1994,michael_schumacher,damon_hill
1997,villeneuve,michael_schumacher
1999,hakkinen,irvine


In [ ]:
# dataviz3
f1cube.visualize('Base vs. Simulation - Brazil 2008')

In [ ]:
# dataviz4
f1cube.visualize('Base vs. Simulation - 2008 Championship table')

In [207]:
# dataviz5
f1cube.visualize('Base vs. Simulation System 1991 to 2002 - 2008 Championship')

Install the ActiveViam JupyterLab extension to see this widget.